<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%202/Modulo_2_Ejercicio_3_ETL_Transformacion_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Ejercicio 3: Integración y Transformación de Datos con ETL**
## Aplicación de un proceso ETL para consolidación de datos de múltiples fuentes.



## Introducción

En este ejercicio, aprenderemos a realizar un **proceso ETL (Extracción, Transformación y Carga)** utilizando datos provenientes de diferentes fuentes.  
La integración de datos es un paso crucial en cualquier pipeline de análisis, ya que permite consolidar información y prepararla para futuros análisis o modelos de Machine Learning.

### Objetivos del ejercicio:
- Cargar datasets desde distintas fuentes (CSV y JSON).
- Unificar y combinar los datasets utilizando operaciones de **merge** y **join**.
- Crear nuevas columnas a partir de los datos originales.
- Convertir variables categóricas a formato numérico con **one-hot encoding** y **label encoding**.
- Guardar el dataset transformado en un nuevo archivo para futuros análisis.

### Conceptos clave:
- Procesos ETL  
- Integración de datos  
- Transformación de variables  


In [1]:

# Celda 1: Importación de librerías necesarias

import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Configuración de opciones de visualización
pd.set_option("display.max_columns", None)

print("Librerías importadas correctamente.")


Librerías importadas correctamente.



## Descripción de los Datos

Para este ejercicio, utilizaremos dos datasets de control de calidad industrial:
1. **Dataset principal (CSV):** "Control de calidad en empaques de alimentos".  
2. **Dataset complementario (JSON):** "Información adicional de proveedores".  

Estos datos nos permitirán realizar un proceso de integración para consolidar información relevante.

### **Fuente de los datasets:**  
- **CSV:** [Control de calidad en empaques de alimentos](https://raw.githubusercontent.com/dtoralg/IE_Calidad_ML/refs/heads/main/Data/control_calidad_empaques.csv)  
- **JSON:** Dataset simulado con información de proveedores  

### **Estructura del dataset principal (CSV):**
| Columna               | Descripción |
|-----------------------|-------------|
| ID_envase            | Identificador único del envase |
| Peso_envase          | Peso del envase en gramos |
| Espesor_material     | Espesor del material en mm |
| Dureza_superficie    | Dureza de la superficie en N/mm² |
| Temperatura_sellado  | Temperatura de sellado en °C |
| Tiempo_prensado      | Tiempo de prensado en segundos |
| Proveedor_material   | Proveedor del material del envase (categoría) |
| Tipo_envase          | Tipo de envase (categoría) |
| Color_material       | Color del material (categoría) |
| Defecto_detectado    | Tipo de defecto detectado en el envase (multiclase) |

### **Estructura del dataset complementario (JSON):**
| Columna               | Descripción |
|-----------------------|-------------|
| Proveedor_material   | Nombre del proveedor |
| País_origen         | País de origen del proveedor |
| Calidad_certificada | Indica si el proveedor tiene certificación de calidad |


In [2]:

# Celda 2: Cargar el dataset principal desde GitHub (CSV)

url_csv = "https://raw.githubusercontent.com/dtoralg/IE_Calidad_ML/refs/heads/main/Data/control_calidad_empaques.csv"
df_csv = pd.read_csv(url_csv)

# Mostrar las primeras filas del dataset
df_csv.head()


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Color_material,Defecto_detectado
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Azul,Rasgado
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Transparente,Rasgado
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Azul,Rasgado
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Blanco,Doble capa
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Azul,Sin defecto


In [3]:

# Celda 3: Crear y cargar el dataset complementario (JSON)

# Definimos un conjunto de datos ficticio en formato JSON para simular la información de proveedores
json_data = '''
[
    {"Proveedor_material": "Proveedor_1", "País_origen": "España", "Calidad_certificada": "Sí"},
    {"Proveedor_material": "Proveedor_2", "País_origen": "Francia", "Calidad_certificada": "No"},
    {"Proveedor_material": "Proveedor_3", "País_origen": "Alemania", "Calidad_certificada": "Sí"},
    {"Proveedor_material": "Proveedor_4", "País_origen": "Italia", "Calidad_certificada": "No"}
]
'''

# Cargar el JSON como un DataFrame
df_json = pd.read_json(json_data)

# Mostrar las primeras filas del dataset
df_json.head()


<ipython-input-3-9c09f73f8e96>:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_json = pd.read_json(json_data)


,Proveedor_material,País_origen,Calidad_certificada
0,Proveedor_1,España,Sí
1,Proveedor_2,Francia,No
2,Proveedor_3,Alemania,Sí
3,Proveedor_4,Italia,No


In [4]:

# Celda 4: Unificación de los datasets

# Realizamos un merge entre el dataset principal y el dataset complementario en base a la columna "Proveedor_material"
df_merged = df_csv.merge(df_json, on="Proveedor_material", how="left")

# Mostrar las primeras filas del dataset unificado
df_merged.head()


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Color_material,Defecto_detectado,País_origen,Calidad_certificada
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Azul,Rasgado,NaN,NaN
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Transparente,Rasgado,Italia,No
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Azul,Rasgado,NaN,NaN
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Blanco,Doble capa,Italia,No
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Azul,Sin defecto,NaN,NaN


In [5]:

# Celda 5: Creación de nuevas columnas

# Crear una columna binaria indicando si el proveedor tiene certificación de calidad
df_merged["Calidad_certificada"] = df_merged["Calidad_certificada"].map({"Sí": 1, "No": 0})

# Extraer información adicional de "Tipo_envase"
df_merged["Tipo_envase_num"] = df_merged["Tipo_envase"].str.extract("(\d+)").astype(float)

print("Nuevas columnas creadas correctamente.")
df_merged.head()


Nuevas columnas creadas correctamente.


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Color_material,Defecto_detectado,País_origen,Calidad_certificada,Tipo_envase_num
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Azul,Rasgado,NaN,NaN,4.0
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Transparente,Rasgado,Italia,0.0,1.0
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Azul,Rasgado,NaN,NaN,1.0
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Blanco,Doble capa,Italia,0.0,2.0
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Azul,Sin defecto,NaN,NaN,4.0


In [6]:

# Celda 6: Transformación de variables categóricas

# Aplicar Label Encoding a la columna "País_origen"
label_encoder = LabelEncoder()
df_merged["País_origen_encoded"] = label_encoder.fit_transform(df_merged["País_origen"].astype(str))

# Aplicar One-Hot Encoding a la columna "Color_material"
df_one_hot = pd.get_dummies(df_merged, columns=["Color_material"], drop_first=True)

print("Transformaciones aplicadas correctamente.")
df_one_hot.head()


Transformaciones aplicadas correctamente.


,ID_envase,Peso_envase,Espesor_material,Dureza_superficie,Temperatura_sellado,Tiempo_prensado,Proveedor_material,Tipo_envase,Defecto_detectado,País_origen,Calidad_certificada,Tipo_envase_num,País_origen_encoded,Color_material_Blanco,Color_material_Transparente,Color_material_Verde
0,1,52.483571,0.251530,131.236814,166.001783,8.035426,Proveedor_7,Envase_4,Rasgado,NaN,NaN,4.0,4,False,False,False
1,2,49.308678,0.142232,98.115434,185.478228,4.851902,Proveedor_4,Envase_1,Rasgado,Italia,0.0,1.0,3,False,True,False
2,3,53.238443,0.228772,141.912131,178.792377,4.524469,Proveedor_6,Envase_1,Rasgado,NaN,NaN,1.0,4,False,False,False
3,4,57.615149,0.169038,72.227237,177.167362,5.515039,Proveedor_4,Envase_2,Doble capa,Italia,0.0,2.0,3,True,False,False
4,5,48.829233,0.183630,93.146984,187.675692,4.936259,Proveedor_9,Envase_4,Sin defecto,NaN,NaN,4.0,4,False,False,False


In [7]:

# Celda 7: Guardar el dataset transformado en un archivo CSV

output_file = "control_calidad_empaques_transformado.csv"
df_one_hot.to_csv(output_file, index=False)

print(f"Dataset transformado guardado como {output_file}.")

# Puedes haciendo click en 'Archivos' (icono de carpeta en la barra lateral izquierda de Colab)

Dataset transformado guardado como control_calidad_empaques_transformado.csv.



## Conclusiones

En este ejercicio hemos realizado un proceso **ETL (Extracción, Transformación y Carga)** con datos industriales.

### Puntos clave:
- Se integraron datos de **múltiples fuentes** (CSV y JSON) mediante operaciones de **merge**.
- Se crearon **nuevas columnas** para enriquecer el dataset.
- Se aplicaron técnicas de **transformación de variables categóricas**, incluyendo **label encoding** y **one-hot encoding**.
- Se generó un **dataset final transformado** listo para futuros análisis.

### Posibles mejoras:
- Implementar validaciones para la limpieza de datos antes de la integración.
- Aplicar estrategias de manejo de valores faltantes en datasets más complejos.
- Explorar la normalización o escalado de variables para mejorar la preparación de datos.
